# 이 쥬피터 노트북은 Keras의 기능을 정확히 알고 이해하기 위해 테스트하기 위해 생성되었습니다.

# Embedding test

In [1]:
from keras.layers import Embedding
from keras.models import Sequential
from keras.constraints import MinMaxNorm
from keras.initializers import RandomUniform

import numpy as np

Using TensorFlow backend.


In [7]:
input_array = np.random.randint(1000, size=(1, 10))
print(input_array)
input_array[:, -1] = 0
print(input_array)

[[306 195 913 815 717 639 377 598 655 513]]
[[306 195 913 815 717 639 377 598 655   0]]


In [8]:
model = Sequential()
model.add(Embedding(1000, 64, 
#                     embeddings_constraint=MinMaxNorm(min_value=0.0, max_value=1.0, axis=1),
                    embeddings_initializer=RandomUniform(minval=0.0, maxval=1.0),
                    input_length=10))
# the model will take as input an integer matrix of size (batch, input_length).
# the largest integer (i.e. word index) in the input should be
# no larger than 999 (vocabulary size).
# now model.output_shape == (None, 10, 64), where None is the batch dimension.

model.compile('rmsprop', 'mse')
output_array = model.predict(input_array)
assert output_array.shape == (1, 10, 64)

In [9]:
output_array

array([[[1.57068253e-01, 4.14005280e-01, 1.11938715e-02, 3.28269958e-01,
         1.21910691e-01, 1.79611564e-01, 4.29499149e-03, 9.83291149e-01,
         3.01469564e-02, 6.95359588e-01, 3.02027464e-01, 8.26856136e-01,
         4.55428720e-01, 1.38051748e-01, 5.47891140e-01, 9.10370588e-01,
         5.50666690e-01, 7.89451599e-01, 5.75811744e-01, 9.44201469e-01,
         6.38316870e-01, 5.87837934e-01, 9.21189666e-01, 2.90876031e-01,
         4.32536602e-01, 9.46827531e-01, 1.53092384e-01, 6.94501400e-02,
         7.59752750e-01, 3.22652102e-01, 7.37296224e-01, 8.32915783e-01,
         1.84330821e-01, 3.42403889e-01, 3.30157280e-02, 5.91504097e-01,
         2.00174928e-01, 2.98762679e-01, 9.65146899e-01, 3.56773973e-01,
         3.71452570e-01, 7.48621702e-01, 3.08681846e-01, 1.75964236e-01,
         3.73154759e-01, 6.45201445e-01, 3.56435776e-03, 3.95624876e-01,
         2.74600625e-01, 4.41686392e-01, 9.06535029e-01, 6.45145178e-01,
         1.73165798e-01, 4.34506893e-01, 8.67923260

# Lambda Test

In [2]:
from keras.models import Sequential, Model
from keras.layers import Lambda, Input, Concatenate, TimeDistributed, Dense
import keras.backend.tensorflow_backend as K
import numpy as np

In [3]:
seq_length = 3
latent_dim = 2
input_array = np.random.randint(10, size=(1, seq_length, latent_dim))

In [4]:
input_array

array([[[3, 4],
        [4, 7],
        [0, 1]]])

In [6]:
axis = 2
def antirectifier(x):
    
    return x ** 2

def antirectifier_output_shape(x):
    return tuple((x[0], ) + x[1:axis] + (seq_length, ) + x[axis:])

In [7]:
model = Sequential()
model.add(Lambda(antirectifier, antirectifier_output_shape))
# model.add(Lambda(antirectifier))
model.compile(optimizer='rmsprop', loss='mse')


output_array = model.predict(input_array)

In [8]:
for a, b in zip(input_array[0], output_array[0]):
    print(a, '->',  b)

[2 2] -> [4. 4.]
[6 3] -> [36.  9.]
[7 1] -> [49.  1.]


# Repeat Vector Layer

In [5]:
from keras.models import Sequential, Model
from keras.layers import Lambda, Input, Concatenate, TimeDistributed, Dense
import keras.backend.tensorflow_backend as K
import numpy as np

In [6]:
enc_seq_length = 3
latent_dim = 2

enc_input_array = np.random.randint(10, size=(1, enc_seq_length, latent_dim))
dec_seq_length = 2
dec_input_array = np.random.randint(10, size=(1, dec_seq_length, latent_dim))

In [8]:
print(enc_input_array)
print(dec_input_array)

[[[5 1]
  [9 2]
  [4 2]]]
[[[8 5]
  [0 8]]]


In [7]:
def RepeatVectorLayer(rep, axis):
#     print(axis)
    return lambda x: K.repeat_elements(K.expand_dims(x, axis), rep, axis)
#     return K.repeat_elements(K.expand_dims(x, axis), rep, axis)

In [7]:
enc_inputs = Input(shape=(enc_seq_length, latent_dim))
dec_inputs = Input(shape=(dec_seq_length, latent_dim))

repeat_d = Lambda(RepeatVectorLayer(enc_seq_length, 2))(dec_inputs)
print(repeat_d.get_shape())
repeat_e = Lambda(RepeatVectorLayer(dec_seq_length, 1))(enc_inputs)
print(repeat_e.get_shape())
outputs = Concatenate()([repeat_d, repeat_e])
model = Model(inputs=[enc_inputs, dec_inputs], outputs=outputs)
model.compile(optimizer='rmsprop', loss='mse')
output_array = model.predict([enc_input_array, dec_input_array])

(?, 2, 3, 2)
(?, 2, 3, 2)


In [8]:
for a, b, c in zip(enc_input_array[0], dec_input_array[0], output_array[0]):
    print(a, b, '->',  c)

[7 6] [0 9] -> [[0. 9. 7. 6.]
 [0. 9. 7. 6.]
 [0. 9. 2. 3.]]
[7 6] [5 2] -> [[5. 2. 7. 6.]
 [5. 2. 7. 6.]
 [5. 2. 2. 3.]]


In [12]:
print(input_array.shape)
print(output_array.shape)

(1, 3, 2)
(1, 3, 3, 4)


# RepeatVector from Keras

In [72]:
from keras.layers import RepeatVector, Reshape, Permute, TimeDistributed, concatenate, merge

In [95]:
inputs = Input(shape=(seq_length, latent_dim))
reshaped_d_inputs = Reshape((inputs.get_shape()[1] * inputs.get_shape()[2], ))(inputs)
print(reshaped_d_inputs.get_shape())
repeat_d_repeat = RepeatVector(seq_length)(reshaped_d_inputs)
repeat_d = Reshape((seq_length, inputs.get_shape()[1], inputs.get_shape()[2]))(repeat_d_repeat)
print(repeat_d.get_shape())


permuted_e = Permute((2, 1))(inputs)
reshaped_e_inputs = Reshape((inputs.get_shape()[1] * inputs.get_shape()[2], ))(permuted_e)
permuted_e_repeat = RepeatVector(seq_length)(reshaped_e_inputs)
repeat_e = Reshape((seq_length, inputs.get_shape()[2], inputs.get_shape()[1]))(permuted_e_repeat)
repeat_e = Permute((1, 3, 2))(repeat_e)
print(repeat_e.get_shape())

outputs = Concatenate(-1)([repeat_d, repeat_e])
model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='rmsprop', loss='mse')
output_array = model.predict(input_array)

(?, 6)
(?, 3, 3, 2)
(?, 3, 3, 2)


TypeError: unhashable type: 'Dimension'

In [120]:
inputs = Input(shape=(seq_length, latent_dim))
repeat_d = K.repeat(inputs, 2)
print(repeat_d.get_shape())
# # repeat_d = TimeDistributed(repeat_d)


# reshaped_e_inputs = Reshape((inputs.get_shape()[1] * inputs.get_shape()[2], ))(inputs)
# print(reshaped_e_inputs.get_shape())
# repeat_e =  RepeatVector(seq_length)(reshaped_e_inputs)
# print(repeat_e.get_shape())
# # repeat_e = Reshape((seq_length, seq_length, latent_dim))(repeat_e)

# # outputs = Concatenate()([repeat_d, repeat_e])
# outputs = np.concatenate([repeat_d, repeat_e])

model = Model(inputs=inputs, outputs=repeat_d)
model.compile(optimizer='rmsprop', loss='mse')
output_array = model.predict(input_array)

AssertionError: 

In [82]:
for a, b in zip(input_array[0], output_array[0]):
    print(a, '->',  b)

[2 2] -> [[[2. 2.]
  [6. 3.]
  [7. 1.]]

 [[2. 2.]
  [6. 3.]
  [7. 1.]]

 [[2. 2.]
  [6. 3.]
  [7. 1.]]]


# Attention layer test

In [10]:
inputs = Input(shape=(seq_length, latent_dim))
repeat_d = Lambda(RepeatVectorLayer(seq_length, 2))(inputs)
repeat_e = Lambda(RepeatVectorLayer(seq_length, 1))(inputs)
concat_v = Concatenate()([repeat_d, repeat_e])
dense_score_layer = TimeDistributed(Dense(latent_dim // 2, activation='tanh'))
dense_score = dense_score_layer(concat_v)
outputs = dense_score

model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='rmsprop', loss='mse')
print(model.summary())
output_array = model.predict(input_array)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3, 2)         0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 3, 3, 2)      0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 3, 3, 2)      0           input_1[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 3, 3, 4)      0           lambda_1[0][0]                   
                                                                 lambda_2[0][0]                   
__________

In [11]:
layer_outputs = list()
for idx, l in enumerate(model.layers):
    if idx == 0:
        continue
    layer_outputs.append(l.output)
get_3rd_layer_output = K.function([model.layers[0].input],
                                  layer_outputs)
layer_output = get_3rd_layer_output([input_array])

In [12]:
print()
for l_output in layer_output:
    print(l_output[0][0])
    print('')


[[3. 4.]
 [3. 4.]
 [3. 4.]]

[[3. 4.]
 [4. 7.]
 [0. 1.]]

[[3. 4. 3. 4.]
 [3. 4. 4. 7.]
 [3. 4. 0. 1.]]

[[-0.95543253]
 [-0.8898261 ]
 [-0.3718664 ]]



In [13]:
layer_output[0][0][0]

array([[3., 4.],
       [3., 4.],
       [3., 4.]], dtype=float32)